In [1]:
# ゼロから作るDeep Learningのコードを書いてみるところ
class Variable:
    def __init__(self, data):
        self.data=data

import numpy as np
data = np.array(1.0)
x = Variable(data)
print(x.data)

# xに新しいデータを代入
x.data = np.array(2.0)
print(x.data)


class Function:
    def __call__(self, input):
        x=input.data #データの取り出し処理
        y=x**2
        output=Variable(y) #Variableとして返す
        return output

x = Variable(np.array(10))
f = Function()
y = f(x)
print(type(y))
print(y.data)




1.0
2.0
<class '__main__.Variable'>
100


In [2]:
# Functionクラスは今後様々な関数を追加することになるので、Functionクラスを基底クラスとして全ての関数に関する共通機能を持つように実装する。
# 具体的な関数はFunctionクラスを継承したクラスで実装する。
class Function:
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(y) 
        return output

    def forward(self,x):
        raise NotImplementedError()

# 次にForwardクラスを継承して、入力された値を２乗するクラスを実装してみる。
class Square(Function):
    def forward(self,x):
        return x ** 2

x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

class Exp(Function):
    def forward(self,x):
        return np.exp(x)

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)




<class '__main__.Variable'>
100
1.648721270700128


In [ ]:
def numerical_diff(f,x,eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

f = Square ()
x = Variable(np.array(2.0))
dy = numerical_diff(f,x)
print(dy)


def f(x):
    A = Square()
    B = EXP()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f,x)
print(dy)



In [ ]:
class Variable:
    def __init__(self,data):
        # data(通常の値)のインスタンス変数を定義。
        self.data = data
        # grad(微分した値)のインスタンス変数を定義。0で初期化
        self.grad = None

class Function:
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(y) 
        self.input = input #入力された変数を覚えさせる。
        return output

    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    #逆伝播のためのbackwordメソッドを追加。
    def backward(self,gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self,x):
        y = np.exp(x)
        return y
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [13]:
# Step 7

class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        # 変数を作成した関数を記憶するための変数
        self.creator = None

    def set_creator(self,func):
        self.creator = func

class Function:
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(y) 
        output.set_creator(self) #出力変数に生みの親を覚えさせる
        self.input = input #入力された変数を覚えさせる。
        self.output = output #出力も覚える
        return output

    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    #逆伝播のためのbackwordメソッドを追加。
    def backward(self,gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self,x):
        y = np.exp(x)
        return y
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

y.grad=np.array(1.0)
C = y.creator
b = C.input
b.grad = C.backward(y.grad)


B = b.creator
a = B.input
a.grad = B.backward(b.grad)

A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)




3.297442541400256


In [20]:
# Step 7 その2

class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        # 変数を作成した関数を記憶するための変数
        self.creator = None

    def set_creator(self,func):
        self.creator = func
    
    def backward(self):
        f = self.creator # 関数取得
        if f is not None:
            x = f.input # 関数の入力取得
            x.grad = f.backward(self.grad) # 関数のbackwardメソッドを呼ぶ。
            x.backward() # 自分より一つ前の変数のbackwardメソッドを呼ぶ。（再帰）


class Function:
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(y) 
        output.set_creator(self) #出力変数に生みの親を覚えさせる
        self.input = input #入力された変数を覚えさせる。
        self.output = output #出力も覚える
        return output

    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    #逆伝播のためのbackwordメソッドを追加。
    def backward(self,gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self,x):
        y = np.exp(x)
        return y
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

# 逆伝播
y.grad =np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [49]:
# Step8,9 backwardのloopへの変換など

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        # 変数を作成した関数を記憶するための変数
        self.creator = None

    def set_creator(self,func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 関数取得
            x, y = f.input, f.output # 関数の入出力を取得。
            x.grad = f.backward(y.grad) # backwardメソッドを呼ぶ。
            if x.creator is not None:
                funcs.append(x.creator) #１つ前の関数をリストに追加。
                
class Function:
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(as_array(y)) 
        output.set_creator(self) #出力変数に生みの親を覚えさせる
        self.input = input #入力された変数を覚えさせる。
        self.output = output #出力も覚える
        return output

    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    #逆伝播のためのbackwordメソッドを追加。
    def backward(self,gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self,x):
        y = np.exp(x)
        return y
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


x = Variable(np.array(1.0))
# x = Variable(None)
# x = Variable(1.0)
y = square(exp(square(x)))

# # 逆伝播
y.backward()
print(x.grad)

29.556224395722598


In [2]:
# Step10 テストを行う。
import numpy as np
import unittest

def numerical_diff(f,x,eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) /(2 * eps)

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
    def test_gradient_check(self):
        x = Variable(np.random.rand(1)) # ランダムな入力値を生成
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)








In [5]:
# Step11

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        # 変数を作成した関数を記憶するための変数
        self.creator = None

    def set_creator(self,func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 関数取得
            x, y = f.input, f.output # 関数の入出力を取得。
            x.grad = f.backward(y.grad) # backwardメソッドを呼ぶ。
            if x.creator is not None:
                funcs.append(x.creator) #１つ前の関数をリストに追加。


#引数と戻り値をリストに変更                
class Function:
    def __call__(self, inputs):
        #　リスト内包表記でinputsの各要素xのリストxsを作成。
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        # outputも同様にリスト内包表記で実装
        outputs = [Variable(as_array(y)) for y in ys] 
        for output in outputs:
            output.set_creator(self) #出力変数に生みの親を覚えさせる
        self.inputs = inputs #入力された変数を覚えさせる。
        self.outputs = outputs #出力も覚える
        return outputs

    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    #逆伝播のためのbackwordメソッドを追加。
    def backward(self,gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self,x):
        y = np.exp(x)
        return y
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

class Add(Function):
    def forward(self,xs):
        x0, x1 = xs
        y = x0 + x1
        # タプルを返すためにこの形
        return (y,)

def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

def numerical_diff(f,x,eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) /(2 * eps)

# x = Variable(np.array(1.0))
# x = Variable(None)
# x = Variable(1.0)
# y = square(exp(square(x)))

# # 逆伝播
#y.backward()
#print(x.grad)

xs = [Variable(np.array(2)),Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0] #ysはタプル
print(y.data)

5


In [7]:
# Step12,13 引数の改善

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        # 変数を作成した関数を記憶するための変数
        self.creator = None

    def set_creator(self,func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 関数取得
            # 出力変数outputsの微分をリストにまとめる。
            gys = [output.grad for output in f.outputs]
            # fの逆伝播を呼び出す。
            gxs = f.backward(*gys)
            # gxsがタプルでない時タプルへ変換。
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator) #１つ前の関数をリストに追加。
 


#引数と戻り値をリストに変更                
class Function:
    def __call__(self, *inputs):
        #　リスト内包表記でinputsの各要素xのリストxsを作成。
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #アスタリスクをつけて渡すことでリストをアンパックして関数に渡す
        if not isinstance(ys, tuple):
            ys = (ys,)
        # outputも同様にリスト内包表記で実装
        outputs = [Variable(as_array(y)) for y in ys] 
        for output in outputs:
            output.set_creator(self) #出力変数に生みの親を覚えさせる
        self.inputs = inputs #入力された変数を覚えさせる。
        self.outputs = outputs #出力も覚える
        return outputs if len(outputs) > 1 else outputs[0]
        

    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    #逆伝播のためのbackwordメソッドを追加。
    def backward(self,gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self,x):
        y = np.exp(x)
        return y
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

class Add(Function):
    def forward(self,x0, x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        return gy, gy



def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

def numerical_diff(f,x,eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) /(2 * eps)

def add(x0, x1):
    return Add()(x0, x1)

# x = Variable(np.array(1.0))
# x = Variable(None)
# x = Variable(1.0)
# y = square(exp(square(x)))

# # 逆伝播
#y.backward()
#print(x.grad)

x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)



5


In [4]:
# Step14 同じ変数が繰り返し出る場合に向けた対応
# Step15,16 逆伝播の世代を正しく設定。

import numpy as np
import unittest

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        # 変数を作成した関数を記憶するための変数
        self.creator = None
        # 世代の初期値
        self.generation = 0

    def set_creator(self,func):
        self.creator = func
        #出力は親となる関数より1世代大きくなる。
        self.generation = func.generation + 1
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        # 関数の中でしか関数を使わない時
        # 親となる関数の中で指定している変数にアクセスする必要がある。（funcsやseen_set）
        #　上記2点を満たすため、add_func関数はbackward関数の中に記載される。
        def add_func(f):
            # seen_setを使って、backwardが2回呼ばれるのを防ぐ。
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                # ここでは全ての要素をソートしているが、より良い方法がある。例えば優先度つきキュー。Pythonではheadqとか。
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop() # 関数取得
            # 出力変数outputsの微分をリストにまとめる。
            gys = [output.grad for output in f.outputs]
            # fの逆伝播を呼び出す。
            gxs = f.backward(*gys)
            # gxsがタプルでない時タプルへ変換。
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_func(x.creator) #１つ前の関数をリストに追加。
    
    def cleargrad(self):
        self.grad = None
 


#引数と戻り値をリストに変更                
class Function:
    def __call__(self, *inputs):
        #　リスト内包表記でinputsの各要素xのリストxsを作成。
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #アスタリスクをつけて渡すことでリストをアンパックして関数に渡す
        if not isinstance(ys, tuple):
            ys = (ys,)
        # outputも同様にリスト内包表記で実装
        outputs = [Variable(as_array(y)) for y in ys] 
        #関数の世代は入力の最大と等しくなる。
        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self) #出力変数に生みの親を覚えさせる
        self.inputs = inputs #入力された変数を覚えさせる。
        self.outputs = outputs #出力も覚える
        return outputs if len(outputs) > 1 else outputs[0]
        

    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    #逆伝播のためのbackwordメソッドを追加。
    def backward(self,gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self,x):
        y = np.exp(x)
        return y
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

class Add(Function):
    def forward(self,x0, x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        return gy, gy



def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

def numerical_diff(f,x,eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) /(2 * eps)

def add(x0, x1):
    return Add()(x0, x1)

# x = Variable(np.array(1.0))
# x = Variable(None)
# x = Variable(1.0)
# y = square(exp(square(x)))

# # 逆伝播
#y.backward()
#print(x.grad)

x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

x.cleargrad
x = Variable(np.array(3.0))
y = add(add(x,x),x)
y.backward()
print(x.grad)

x.cleargrad
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a),square(a))
y.backward()
print(y.data)
print(x.grad)

2.0
3.0
32.0
64.0


In [12]:
# Step17　メモリ管理と循環参照対策（弱参照）
# ただし、Jupyter上などだとその処理系が参照していることがあるのでゼロにはならなくなる。
# Step18 不要な微分の不保持と逆伝播無効モード実装のためのConfigクラス

import numpy as np
import unittest
import weakref
import contextlib

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        # 変数を作成した関数を記憶するための変数
        self.creator = None
        # 世代の初期値
        self.generation = 0

    def set_creator(self,func):
        self.creator = func
        #出力は親となる関数より1世代大きくなる。
        self.generation = func.generation + 1
    
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        # 関数の中でしか関数を使わない時
        # 親となる関数の中で指定している変数にアクセスする必要がある。（funcsやseen_set）
        #　上記2点を満たすため、add_func関数はbackward関数の中に記載される。
        def add_func(f):
            # seen_setを使って、backwardが2回呼ばれるのを防ぐ。
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                # ここでは全ての要素をソートしているが、より良い方法がある。例えば優先度つきキュー。Pythonではheadqとか。
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop() # 関数取得
            # 出力変数outputsの微分をリストにまとめる。弱参照処理に合わせて()を追記。
            gys = [output().grad for output in f.outputs]
            # fの逆伝播を呼び出す。
            gxs = f.backward(*gys)
            # gxsがタプルでない時タプルへ変換。
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_func(x.creator) #１つ前の関数をリストに追加。

        if not retain_grad:
            for y in f.outputs:
                y().grad = None #yはweakrefなので()がいる。
    def cleargrad(self):
        self.grad = None
 


#引数と戻り値をリストに変更                
class Function:
    def __call__(self, *inputs):
        #　リスト内包表記でinputsの各要素xのリストxsを作成。
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #アスタリスクをつけて渡すことでリストをアンパックして関数に渡す
        if not isinstance(ys, tuple):
            ys = (ys,)
        # outputも同様にリスト内包表記で実装
        outputs = [Variable(as_array(y)) for y in ys] 
        # 逆伝播有効モード実行時のみ実施。
        if Config.enable_backprop:
            #関数の世代は入力の最大と等しくなる。
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self) #出力変数に生みの親を覚えさせる
        self.inputs = inputs #入力された変数を覚えさせる。
        self.outputs = [weakref.ref(output) for output in outputs] #出力も覚える。弱参照で。
        return outputs if len(outputs) > 1 else outputs[0]
        
    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Config:
    enable_backprop = True

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    #逆伝播のためのbackwordメソッドを追加。
    def backward(self,gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self,x):
        y = np.exp(x)
        return y
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

class Add(Function):
    def forward(self,x0, x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        return gy, gy

#using_Config関数をwith構文で使うためのデコレータ
@contextlib.contextmanager
# nameにはConfigの属性名（class属性名を指定。）
def using_config(name, value):
    # 前処理（古い値を取り出して新しいいvalueを設定。）
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    # 本処理(with構文の中の処理)
    try:
        yield 
    #後処理
    finally:
        #valueを元に戻す
        setattr(Config, name, old_value)

# with構文の後にusing...と何回も書くのは面倒なので
def no_grad():
    return using_config('enable_backprop', False)

def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

def numerical_diff(f,x,eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) /(2 * eps)

def add(x0, x1):
    return Add()(x0, x1)


with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)


In [37]:
# Step19  変数を使いやすくする
# Step20,21,22 演算子のオーバーロード
import numpy as np
import unittest
import weakref
import contextlib

class Variable:
    #ndarrayが前に来たときの演算子を用いた計算でndarrayより優先してメソッドを読んでもらうための設定
    __array_priority__ = 200
    # 変数に名前をつけれるように改良
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.name = name
        self.grad = None
        # 変数を作成した関数を記憶するための変数
        self.creator = None
        # 世代の初期値
        self.generation = 0

    def set_creator(self,func):
        self.creator = func
        #出力は親となる関数より1世代大きくなる。
        self.generation = func.generation + 1
    
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        # 関数の中でしか関数を使わない時
        # 親となる関数の中で指定している変数にアクセスする必要がある。（funcsやseen_set）
        #　上記2点を満たすため、add_func関数はbackward関数の中に記載される。
        def add_func(f):
            # seen_setを使って、backwardが2回呼ばれるのを防ぐ。
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                # ここでは全ての要素をソートしているが、より良い方法がある。例えば優先度つきキュー。Pythonではheadqとか。
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop() # 関数取得
            # 出力変数outputsの微分をリストにまとめる。弱参照処理に合わせて()を追記。
            gys = [output().grad for output in f.outputs]
            # fの逆伝播を呼び出す。
            gxs = f.backward(*gys)
            # gxsがタプルでない時タプルへ変換。
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_func(x.creator) #１つ前の関数をリストに追加。

        if not retain_grad:
            for y in f.outputs:
                y().grad = None #yはweakrefなので()がいる。
    def cleargrad(self):
        self.grad = None
 
    # Variableインスタンスをndarrayのインスタンスのように見せるための実装。(必要に応じて幾つでも設定可能。)
    
    # インスタンス変数としてアクセスするためのデコレータ
    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size
    
    @property
    def shape(self):
        return self.data.dtype

    # PYTHOのlen関数が使えるように拡張
    # __len__という特殊メソッドを実装。
    def __len__(self):
        return len(self.data)

    # print関数が使えるように拡張
    # 必要なら文字列の中身を変更可能。
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'
    
    #def __mul__(self, other):
    #    return mul(self, other)


#引数と戻り値をリストに変更                
class Function:
    def __call__(self, *inputs):
        # inputsの各要素をVariableインスタンスに変換（Variableインスタンスではない場合）
        inputs = [as_variable(x) for x in inputs]
        #　リスト内包表記でinputsの各要素xのリストxsを作成。
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #アスタリスクをつけて渡すことでリストをアンパックして関数に渡す
        if not isinstance(ys, tuple):
            ys = (ys,)
        # outputも同様にリスト内包表記で実装
        outputs = [Variable(as_array(y)) for y in ys] 
        # 逆伝播有効モード実行時のみ実施。
        if Config.enable_backprop:
            #関数の世代は入力の最大と等しくなる。
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self) #出力変数に生みの親を覚えさせる
        self.inputs = inputs #入力された変数を覚えさせる。
        self.outputs = [weakref.ref(output) for output in outputs] #出力も覚える。弱参照で。
        return outputs if len(outputs) > 1 else outputs[0]
        
    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Config:
    enable_backprop = True

class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    #逆伝播のためのbackwordメソッドを追加。
    def backward(self,gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self,x):
        y = np.exp(x)
        return y
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

class Add(Function):
    def forward(self,x0, x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        return gy, gy

class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

class Neg(Function):
    def forward(self, x):
        return -x
    def backward(self, gy):
        return -gy

class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y
    def backward(self, gy):
        return gy, -gy

class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy /x1
        gx1 = gy * (- x0 / x1 ** 2 )
        return gx0, gx1

class Pow(Function):
    def __init__(self, c):
            self.c = c

    def forward(self,x):
        y = x ** self.c
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = x * x ** (c - 1) * gy
        return gx


def as_array(x):

    if np.isscalar(x):
        return np.array(x)
    return x

# ndarrayインスタンスをVaraibleインスタンスに変換
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

#using_Config関数をwith構文で使うためのデコレータ
@contextlib.contextmanager
# nameにはConfigの属性名（class属性名を指定。）
def using_config(name, value):
    # 前処理（古い値を取り出して新しいいvalueを設定。）
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    # 本処理(with構文の中の処理)
    try:
        yield 
    #後処理
    finally:
        #valueを元に戻す
        setattr(Config, name, old_value)

# with構文の後にusing...と何回も書くのは面倒なので
def no_grad():
    return using_config('enable_backprop', False)

def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

def numerical_diff(f,x,eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) /(2 * eps)

def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1) 

def neg(x):
    return Neg()(x)

def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)
def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)

def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0)

def pow(x,c):
    return Pow(c)(x)

Variable.__mul__ = mul
#計算の右の項に来たときに呼び出すもの
Variable.__rmul__ = mul
Variable.__add__ = add
Variable.__radd__ = add

Variable.__neg__ = neg
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
Variable.__pow__ = pow


with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))
#y = add(mul(a,b),c)

y = a * b + c
y.backward()
print(y)
print(a.grad)
print(b.grad)

y = a + np.array(3.0)
print(y)

x = Variable(np.array(1.0))
y = (x + 3) ** 2
y.backward()
print(y)
print(x.grad)


variable(7.0)
2.0
3.0
variable(6.0)
variable(16.0)
16.0
